In [9]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import text
from sqlalchemy import Column, Integer, String, Float , Boolean
import pandas as pd

In [5]:
#Base = declarative_base()

#class moives(Base):
#    __tablename__ = 'movies'
#    adult = Column(Boolean)
#    original_title = Column(String())
#   release_date = Column(String())
#    imdb_id = Column(String(), primary_key=True)
#    popularity = Column(String())


In [10]:
engine = create_engine("sqlite:///movies.sqlite")
conn = engine.connect()

In [11]:
#insert movies data
data = pd.read_csv('./CleanedCSV/reduced_Clean.csv')
data.to_sql('movies', engine)

45332

In [14]:
#insert goof data
data2 = pd.read_csv('./CleanedCSV/goofsnocommas.csv')
data2.to_sql('goofs', engine)

40001

In [12]:

  
# establish the connection with the engine object
with engine.connect() as conn:
    
    # let's select the column credit_history
    # from the loan data table
    result = conn.execute(text("SELECT imdb_id FROM movies"))
      
    # print the result
    for row in result:
        print(row.imdb_id)

tt0114709
tt0113497
tt0113228
tt0114885
tt0113041
tt0113277
tt0114319
tt0112302
tt0114576
tt0113189
tt0112346
tt0112896
tt0112453
tt0113987
tt0112760
tt0112641
tt0114388
tt0113101
tt0112281
tt0113845
tt0113161
tt0112722
tt0112401
tt0114168
tt0113627
tt0114057
tt0114011
tt0114117
tt0112682
tt0115012
tt0112792
tt0114746
tt0114952
tt0112431
tt0112637
tt0112818
tt0112286
tt0113442
tt0112697
tt0112749
tt0114279
tt0112819
tt0114272
tt0113855
tt0114681
tt0113347
tt0114369
tt0114148
tt0114916
tt0114814
tt0109950
tt0113819
tt0110299
tt0112499
tt0113158
tt0113541
tt0113321
tt0110877
tt0112714
tt0113419
tt0116260
tt0113862
tt0116126
tt0118002
tt0115683
tt0116839
tt0114753
tt0113149
tt0113118
tt0116367
tt0113010
tt0113537
tt0113828
tt0115644
tt0115676
tt0114367
tt0113973
tt0112744
tt0116731
tt0112445
tt0114660
tt0112379
tt0114039
tt0113612
tt0112365
tt0118158
tt0116151
tt0115697
tt0113972
tt0113490
tt0117002
tt0114825
tt0115639
tt0115759
tt0113403
tt0113247
tt0111173
tt0113283
tt0115907
tt0115734
